# Prepare Train Data

In [1]:
from datetime import datetime, timezone, date
from datetime import timedelta
from pathlib import Path
import pandas as pd
import pyarrow as pa

In [2]:
CONSUMPTION_DATA_PATH = Path("../data/consumption")
CONSUMPTION_NE5_FILE = CONSUMPTION_DATA_PATH / "NE5_Export.csv"
CONSUMPTION_NE7_FILE = CONSUMPTION_DATA_PATH / "NE7_Export.csv"
TRAIN_DATA_PATH = Path('../data/consumption/final_train.parquet')
DATA_PATH_2022 = Path('../data/consumption/2022.parquet')

METEO_PATH = Path("../data/meteoswiss/reh_nzz.csv")
METEO_STATION = "REH"
METEO_TEMP_PARAMETER = "tre200h0"

TIME_FORMAT = "dd.MM.yyyy"
WINDOW_DAYS = 1
START_DATE = date(2010, 1, 1)
END_DATE = date(2022, 1, 1)

In [3]:
#ne5File = spark.read.csv(CONSUMPTION_NE5_FILE, header=True, inferSchema=True, sep=";")
#ne7File = spark.read.csv(CONSUMPTION_NE7_FILE, header=True, inferSchema=True, sep=";")

# consumption = (ne5File.alias("ne5")
#                       .join(ne7File.alias("ne7"), "Date")
#                       .withColumn("NE5Consumption", f.expr("ne5.Value"))
#                       .withColumn("NE7Consumption", f.expr("ne7.Value"))
#                       .withColumn("Date", f.to_date(f.col("Date"), TIME_FORMAT))
#                       .filter((f.col("Date") >= START_DATE) & (f.col("Date") < END_DATE))
#                       .select("Date", "NE5Consumption", "NE7Consumption")
#               )

consumption = pd.read_csv('https://data.stadt-zuerich.ch/dataset/ewz_stromabgabe_netzebenen_stadt_zuerich/download/ewz_stromabgabe_netzebenen_stadt_zuerich.csv')
consumption['Date'] = pd.to_datetime(consumption['Timestamp'], errors='raise', utc=True)
#consumption = consumption[(consumption.Date >= START_DATE) & (consumption.Date < END_DATE)]
consumption.rename(columns={'Value_NE5': 'NE5Consumption', 'Value_NE7': 'NE7Consumption'}, inplace=True)
consumption = consumption.groupby(consumption.Date.dt.date).sum(numeric_only=True)

In [18]:
"""
meteoData = (spark.read.format("delta")
                         .load("/MeteoSwiss/Measurement/Delta")
                         .filter((f.col("Station") == METEO_STATION) & (f.col("Parameter") == METEO_TEMP_PARAMETER))
                         .withColumn("Date", f.to_date(f.col("TimestampUtc")))
                         .groupBy("Date")
                         .agg(f.avg(f.col("Value")).alias("Temperature"))
            )
"""

meteoData = pd.read_csv(METEO_PATH, encoding='iso-8859-1', sep=';')
meteoData = meteoData[meteoData.abbr == METEO_STATION]
meteoData['Date'] = pd.to_datetime(meteoData['time'], format='%Y%m%d%H%M', utc=True)

# Convert Timezone!
meteoData['Date'] = meteoData['Date'].dt.tz_convert('Europe/Berlin')

meteoData = meteoData.groupby(meteoData.Date.dt.date).agg(Temperature = (METEO_TEMP_PARAMETER, 'mean'))

meteoData.tail()

,Temperature
Date,
2022-10-28,12.520833
2022-10-29,13.154167
2022-10-30,11.292000
2022-10-31,10.362500
2022-11-01,9.200000


In [19]:
data = consumption.join(meteoData, "Date").reset_index(drop=False)

In [20]:
# data.write.parquet("/ConsumptionModel/Data/final_train.parquet")
data[(data.Date >= START_DATE) & (data.Date < END_DATE)].to_parquet(TRAIN_DATA_PATH, engine='pyarrow')

# 2022
data[(data.Date >= date(2022, 1, 1)) & (data.Date < date(2022, 10, 31))].to_parquet(DATA_PATH_2022, engine='pyarrow')


In [10]:
data[(data.Date >= START_DATE) & (data.Date < END_DATE)]

,Date,NE5Consumption,NE7Consumption,Temperature
1,2010-01-01,1.822814e+06,5.115723e+06,2.691667
2,2010-01-02,1.865160e+06,5.371369e+06,-2.000000
3,2010-01-03,1.789316e+06,5.203847e+06,-4.887500
4,2010-01-04,2.541962e+06,6.682725e+06,-6.745833
5,2010-01-05,2.617126e+06,6.872159e+06,-5.587500
...,...,...,...,...
4379,2021-12-27,2.462366e+06,4.957839e+06,5.387500
4380,2021-12-28,2.455435e+06,5.057680e+06,7.383333
4381,2021-12-29,2.441716e+06,5.077009e+06,8.375000
4382,2021-12-30,2.397142e+06,4.920351e+06,12.316667
